<a href="https://colab.research.google.com/github/Awais-mohammad/ADAS/blob/master/Dental_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
abbasseifossadat_dental_radiography_segmentation_path = kagglehub.dataset_download('abbasseifossadat/dental-radiography-segmentation')

print('Data source import complete.')


<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: white; font-size:100%; text-align:left">

<h1 align="center"><font color='#DAA520'>💡 Dental Radiograph Classification :</font></h1>
    


This document provides a comprehensive guide on a **dental image classification** project using deep learning. It covers the steps involved in _data preparation_, _model building_, _training_, and _evaluation_. The project employs TensorFlow and Keras to build and train a convolutional neural network (CNN) to classify dental radiographs into five categories: **Cavity, Fillings, Impacted Tooth, Implant, and Normal**. The guide includes code snippets for data augmentation, model architecture, and evaluation metrics, offering a detailed walkthrough of the entire machine-learning pipeline for dental image classification.

<div class="alert alert-success">  
    <h1 align="center" style="color:darkgoldenrod;">Data Preparation
</h1>  
     
</div>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# Set the image size and batch size
IMG_SIZE = (64, 64)
BATCH_SIZE = 512

# Define the paths to the directories
train_dir = '/kaggle/input/dental-radiography-segmentation/Dental_Radiography/train'
valid_dir = '/kaggle/input/dental-radiography-segmentation/Dental_Radiography/valid'
test_dir = '/kaggle/input/dental-radiography-segmentation/Dental_Radiography/test'

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True
)

# Just rescaling for validation and testing
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directory for train, validation, and test sets
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False
)
# Check class indices
print("Class Indices: ", train_data.class_indices)

2025-01-01 07:20:32.389843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-01 07:20:32.389982: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-01 07:20:32.591222: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 25136 images belonging to 5 classes.
Found 2812 images belonging to 5 classes.
Found 1649 images belonging to 5 classes.
Class Indices:  {'Cavity': 0, 'Fillings': 1, 'Impacted Tooth': 2, 'Implant': 3, 'Normal': 4}


<div class="alert alert-success">  
    <h1 align="center" style="color:darkgoldenrod;">handling Imbalanced Class
</h1>  
     
</div>

In [ ]:
# Extract class labels from training data
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)

class_weights = dict(enumerate(class_weights))


<div class="alert alert-success">  
    <h1 align="center" style="color:darkgoldenrod;">Model Architecture and Training
</h1>  
     
</div>

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


inp = Input(shape=(*IMG_SIZE, 3))

# First convolutional block
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inp)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D((2, 2))(x)

# Second convolutional block
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D((2, 2))(x)

# Third convolutional block
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Dropout(0.2)(x)
x = MaxPooling2D((2, 2))(x)


# Flatten the output
x = Flatten()(x)

# Fully connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer
out = Dense(5, activation='softmax')(x)

# Model definition
model = Model(inputs=inp, outputs=out)

my_callbacks = [
    EarlyStopping(min_delta=0.0001, monitor='loss', patience=15),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=15),
    ModelCheckpoint(filepath='best_model.keras', monitor='val_loss', save_best_only=True)
]

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Model summary
model.summary()

In [ ]:
# Model fitting
history = model.fit(
    train_data,
    epochs=100,
    validation_data=valid_data,
    class_weight=class_weights,
    callbacks=my_callbacks
)

<div class="alert alert-success">  
    <h1 align="center" style="color:darkgoldenrod;">Loading the Model
</h1>  
     
</div>

In [ ]:
# Load the best saved model
best_model = tf.keras.models.load_model('best_model.keras')

<div class="alert alert-success">  
    <h1 align="center" style="color:darkgoldenrod;">Model Evaluation
</h1>  
     
</div>

In [ ]:
import matplotlib.pyplot as plt

# Plot loss and val_loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

# Plot accuracy and val_accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Evaluate and predict using the best model
def evaluate_and_report(model, data, data_type='Validation'):
    preds = model.predict(data)
    pred_classes = np.argmax(preds, axis=1)
    true_classes = data.classes

    # Print classification report
    report = classification_report(true_classes, pred_classes, target_names=data.class_indices.keys())
    print(f'{data_type} Classification Report:\n', report)

    # Plot confusion matrix
    cm = confusion_matrix(true_classes, pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=data.class_indices.keys())
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {data_type} Data')
    plt.show()

In [ ]:
# Evaluate on validation data
evaluate_and_report(best_model, valid_data, data_type='Validation')

In [ ]:
# Evaluate on test data
evaluate_and_report(best_model, test_data, data_type='Test')

In [ ]:
import random

# Predict the classes for the test data
test_preds = model.predict(test_data)
test_pred_classes = np.argmax(test_preds, axis=1)
test_true_classes = test_data.classes


# Display 30 random images from the test data with actual and predicted labels
fig, axes = plt.subplots(6, 5, figsize=(20, 20))
axes = axes.flatten()

for ax in axes:
    idx = random.randint(0, len(test_data.filenames) - 1)
    img = plt.imread(os.path.join(test_dir, test_data.filenames[idx]))
    ax.imshow(img)
    true_label = list(test_data.class_indices.keys())[test_true_classes[idx]]
    pred_label = list(test_data.class_indices.keys())[test_pred_classes[idx]]
    ax.set_title(f"True: {true_label}\nPred: {pred_label}")
    ax.axis('off')

plt.tight_layout()
plt.show()